# 1. Install

In [1]:
!ls

model_detect.ipynb trainOnColab


In [2]:
# https://pytorch.org/get-started/locally/
!pip3 install torch torchvision torchaudio

--- Logging error ---
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/Library/

In [3]:
!git clone https://github.com/ultralytics/yolov5  # clone

Cloning into 'yolov5'...
remote: Enumerating objects: 14302, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 14302 (delta 26), reused 31 (delta 13), pack-reused 14242
Receiving objects: 100% (14302/14302), 13.65 MiB | 5.33 MiB/s, done.
Resolving deltas: 100% (9826/9826), done.


In [4]:
## Change directory to yolov5
%cd yolov5

/Users/prachayakhomduean/Desktop/Project/person_detect/yolov5


In [5]:
!ls

CITATION.cff     classify         models           tutorial.ipynb
CONTRIBUTING.md  data             requirements.txt utils
LICENSE          detect.py        segment          val.py
README.md        export.py        setup.cfg
benchmarks.py    hubconf.py       train.py


In [6]:
## install required library
!pip install -r requirements.txt  # install


[notice] A new release of pip available: 22.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


# 2. Load custom model

In [7]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [8]:
# yolov5s
model = torch.hub.load('/Users/prachayakhomduean/Desktop/Project/person_detect/yolov5', 'custom', path='/Users/prachayakhomduean/Desktop/Project/person_detect/trainOnColab/exp5/weights/best.pt', source='local') 

requirements: YOLOv5 requirement "tqdm>=4.64.0" not found, attempting AutoUpdate...

[notice] A new release of pip available: 22.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

requirements: 1 package updated per /Users/prachayakhomduean/Desktop/Project/person_detect/yolov5/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 v7.0-23-g5dc1ce4 Python-3.8.5 torch-1.11.0 CPU

Fusing layers... 
Model summary: 213 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


## set confidence

In [9]:
model.conf = 0.5  # confidence threshold (0-1)

In [10]:
model.iou = 0.45  # NMS IoU threshold (0-1)  

# 3. Real time VDO test

In [12]:
# the output will be written to output.avi
out = cv2.VideoWriter(
    'outputss3.avi',
    cv2.VideoWriter_fourcc(*'MJPG'),
    15.,
    (640,480))

In [13]:
# on local computer
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

while cap.isOpened():
    ret, frame = cap.read()
    
    # Make detections 
    if frame is None:
        print('variable is None')
    else:
        
        results = model(frame)
        # Number of class
        df = results.pandas().xyxy[0].groupby(['name'])['name'].count()
        sit = 0
        sleep = 0
        stand = 0
        for class_ in df.index:
            if class_ == 'sit':
                sit = df['sit']
            elif class_ == 'sleep':
                sleep = df['sleep']
            elif class_ == 'stand':
                stand = df['stand']

        total = sit+sleep+stand
        number_person = 'Total:'+ str(total)
        number_sit_person = 'sit:'+ str(sit)
        number_sleep_person = 'sleep:'+ str(sleep)
        number_stand_person = 'stand:'+ str(stand)

        # Use putText() method for Inserting text on video
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, 
                    number_person, 
                    (500, 50), 
                    font, 0.8, 
                    (0, 0, 255), 
                    2, 
                    cv2.LINE_4)

        cv2.putText(frame, 
                    number_sit_person, 
                    (500, 80), 
                    font, 0.8, 
                    (0, 0, 255), 
                    2, 
                    cv2.LINE_4)

        cv2.putText(frame, 
                    number_sleep_person, 
                    (500, 110), 
                    font, 0.8, 
                    (0, 0, 255), 
                    2, 
                    cv2.LINE_4)

        cv2.putText(frame, 
                    number_stand_person, 
                    (500, 140), 
                    font, 0.8, 
                    (0, 0, 255), 
                    2, 
                    cv2.LINE_4)

        cv2.imshow('YOLOv5', np.squeeze(results.render()))
        
        # Write the output video 
        out.write(frame.astype('uint8'))

        if cv2.waitKey(3) & 0xFF == ord('q'):
            break
    
# When everything done, release the capture
cap.release()
# and release the output
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1